In [11]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from IPython.display import Image
from keras import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras import datasets, layers, models, losses

In [4]:
dataset_dir = "C:/Users/Hannah/OneDrive/Documents/MIDS/207/Final_Project/mids_207_final_project/"

img_height = 32
img_width = 32
batch_size = 500

# Load training and validation data
train, val = keras.utils.image_dataset_from_directory(
  os.path.join(dataset_dir, "train"),
  validation_split=0.2,
  subset="both",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Using 20000 files for validation.


In [5]:
test = keras.utils.image_dataset_from_directory(
  os.path.join(dataset_dir, "test"),
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [8]:
classes = train.class_names
classes
len(train)

160

## LRN Version (Don't Use)

In [9]:
model_LRN = models.Sequential()
model_LRN.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=(32,32,3)))
model_LRN.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model_LRN.add(layers.Lambda(tf.nn.local_response_normalization))
model_LRN.add(layers.Activation('relu'))
model_LRN.add(layers.MaxPooling2D(3, strides=2))
model_LRN.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model_LRN.add(layers.Lambda(tf.nn.local_response_normalization))
model_LRN.add(layers.Activation('relu'))
model_LRN.add(layers.MaxPooling2D(3, strides=2))
model_LRN.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model_LRN.add(layers.Activation('relu'))
model_LRN.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model_LRN.add(layers.Activation('relu'))
model_LRN.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model_LRN.add(layers.Activation('relu'))
model_LRN.add(layers.Flatten())
model_LRN.add(layers.Dense(4096, activation='relu'))
model_LRN.add(layers.Dropout(0.5))
model_LRN.add(layers.Dense(4096, activation='relu'))
model_LRN.add(layers.Dropout(0.5))
model_LRN.add(layers.Dense(1, activation='sigmoid'))
model_LRN.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 56, 56, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 56, 56, 96)        0         
                                                                 
 activation (Activation)     (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 256)         614656    
                                                      

In [12]:
model_LRN.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")])

In [13]:
historyLRN = model_LRN.fit(
  train,
  validation_data=val,
  epochs=40,
  verbose=0)

historyLRN = pd.DataFrame(historyLRN.history)
display(historyLRN)

KeyboardInterrupt: 

## Batch Normalization Version

In [18]:
keras.backend.clear_session()
model = keras.models.Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.experimental.preprocessing.Resizing(227, 227, interpolation="bilinear"),
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 resizing (Resizing)         (None, 227, 227, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 55, 55, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       6

In [20]:
model.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")])

In [21]:
epochs = 40

history = model.fit(
  train,
  validation_data=val,
  epochs=epochs,
  verbose=3)

history = pd.DataFrame(history.history)
display(history)

Epoch 1/40


KeyboardInterrupt: 